# **深度学习应用于拉曼光谱官能团数量的识别**  
*Deep Learning for Fuctional Group Quantification Using Raman Spectra.*

本课题利用深度学习模型，根据有机分子的拉曼光谱，识别分子中存在的官能团。
## 目录

# 1 配置虚拟环境 
>**如果你已经配置过其他拥有torch的环境，激活该环境后跳到(5)即可。**

(1) 打开终端 
 
(2) 新建环境  
```
conda create -n fgid python=3.10
```
(3) 进入环境
```
conda activate fgid
```
(4) 安装所需包
```
pip install -r requirements.txt
```
(5) 安装rdkit  
rdkit是用来对化学分子进行结构展示、性质计算等操作的开源工具包。我们将使用rdkit展示分子结构式。 
```
pip install rdkit-pypi==2022.9.4
```

# 2 数据准备
## 2.1 定义随机种子

In [ ]:
import numpy as np
import os, torch, random
SEED = 42 # 也可以设置别的数，设置后保持不变即可。

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

## 2.2 数据集的构建

1. 补全官能团的SMARTS，参考[rdkit_practice](../rdkit_practice)：

In [ ]:
label_names = {
        'Alkane':,
        'Alkene':,
        'Alkyne':,
        'Arene':,
        'Haloalkane':,
        'Alcohol':,
        'Aldehyde':,
        'Ketone':,
        'Ester':,
        'Ether':,
}

2. 进行数据的提取和清洗，保存为.pkl或.npz文件：    
**注意标签的元素不再是0和1，而是官能团的个数**；应该需要把标签归一化

In [2]:
data =  # 光谱数据
label =  # 标签，1表示官能团存在，0表示不存在。
smiles = 

## 2.3 数据集的划分
我们采用分层抽样，将数据集划分成数量比为训练:验证:测试=3:1:1的三个子集，保证每种官能团在各个子集里的数量占比与在全集中相同（实际情况下不可能完全均分，所以取整作近似）。

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

shuffle_splits = 1
# 定义分层抽样器1，使测试集占全集20%。test_size=1/(1+1+3)=0.2
str_shuffle_1 = MultilabelStratifiedShuffleSplit(n_splits=shuffle_splits, test_size=0.2, random_state=SEED)
# 定义分层抽样器2，使验证集占剩训练+验证数据的25%。test_size=1/(1+3)=0.25
str_shuffle_2 = MultilabelStratifiedShuffleSplit(n_splits=shuffle_splits, test_size=0.25, random_state=SEED) 

for train_val_index, test_index in str_shuffle_1.split(data, label):
    X_train_val = data[train_val_index]
    X_test = data[test_index]
    y_train_val = label[train_val_index]
    y_test = label[test_index]
    smiles_train_val = smiles[train_val_index]
    smiles_test = smiles[test_index]
for train_index, val_index in str_shuffle_2.split(X_train_val, y_train_val):
    X_train = X_train_val[train_index]
    X_val = X_train_val[val_index]
    y_train = y_train_val[train_index]
    y_val = y_train_val[val_index]
    smiles_train = smiles_train_val[train_index]
    smiles_val = smiles_train_val[val_index]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

# 2.4 构建Dataloader

In [6]:
from torch.utils.data import DataLoader, TensorDataset
def get_dataloader(X, y, batch_size=64):
    '''
    将光谱X和标签y配对并分为len(X)/batch_size个批次，每批次batch_size个光谱-标签对。'''
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    dataset = TensorDataset(X, y)
    # weighted_sampler = sampler.WeightedRandomSampler(sampling_weight, len(sampling_weight))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

train_loader = get_dataloader(X_train, y_train, batch_size=128)
val_loader = get_dataloader(X_val, y_val, batch_size=64)


# 3 训练模型
## 3.1 训练组件设置
(1) 使用gpu加速  
>如果下面代码输出cpu但是你的电脑有gpu，可以参考教程：[选择正确版本的CUDA和PyTorch安装](https://zhuanlan.zhihu.com/p/672526561)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device # 查看在使用的处理器

(2) 损失函数    
这是一个回归任务，用什么损失函数合适呢

In [ ]:
criterion =
optimizer = Adam(params=model.parameters(), lr=learning_rate)

(3) 早停机制  

In [8]:
class EarlyStopping:
    def __init__(self, save_path, patience=5, delta=0):
        """
        Args:
            save_path: 模型参数保存路径.
            patience (int): 监测指标在连续patience次迭代不升高后，自动停止训练。
                            Default: 5
            delta (float): 监测指标增量小于该值时，不保存模型。
                            Default: 0
        """
        self.save_path = save_path
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} / {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        path = os.path.join(self.save_path, 'best_network.pth')
        torch.save(model.state_dict(), path)
        self.val_loss_min = val_loss

## 3.2 设置训练参数和模型初始化

In [9]:
from model import CNN
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from tqdm import tqdm
from time import time

num_epochs = 100 # 最大迭代次数
model_save_path = 'model_path' # 保存模型参数的路径
learning_rate = 2.5e-4 # 学习率
val_threshold = 0.5 # 该阈值用来与模型计算的官能团存在概率作比较，若概率值大于该阈值，认为官能团存在，输出标签的相应维度上元素输出为1，否则为0。
model = CNN(class_num=y_train.shape[1]).to(device)
'''
pos_weight=? 为每类官能团的loss赋予权重，是否可以提高模型性能？'''



## 3.3 训练和验证   
这是一个回归任务，验证和训练的方式也要改变

In [ ]:
val_loss_list = []
train_loss_list = []
val_loss_min = 1

earlystopping = EarlyStopping(save_path=model_save_path)
val_acc_max = 0.5

time_start = time()
for epoch in range(1, num_epochs+1):

# 训练
    train_loss = 0
    model.train()
    train_bar = tqdm(enumerate(train_loader), total=len(train_loader), ncols=50)
    for _, (batch_x, batch_y) in train_bar:
        optimizer.zero_grad()
        outputs = model(batch_x.to(device))
        loss = criterion(outputs, batch_y.to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)
    print(f'Epoch [{epoch}/{num_epochs}], Training Loss: {train_loss}.')

# 验证
    model.eval()
    val_loss = 0
    val_acc = 0
    num_correct = 0
    with torch.no_grad():
        val_bar = tqdm(enumerate(val_loader), total=len(val_loader), ncols=50)  # 创建进度条
        for _, (batch_x, batch_y) in val_bar:
            val_outputs = model(batch_x.to(device))
            loss = criterion(val_outputs, batch_y.to(device))
            optimizer.zero_grad()
            val_loss += loss.item()
            '''下面改成什么呢
            '''
            # predicted_labels = (val_outputs >= val_threshold).float().cpu()
            # num_correct += np.count_nonzero(predicted_labels == batch_y)

    # val_acc = num_correct/(y_val.shape[0]*y_val.shape[1]) 
    # val_loss = val_loss / len(val_loader)
    # print(f'Epoch [{epoch }/{num_epochs}], Validation Accuracy: {val_acc}.')

    if earlystopping:
        earlystopping(val_loss, model)
        if earlystopping.early_stop:
            print("Early stopped.")
            time_end=time()
            break
print(f'Total training time: {(time_end-time_start)//60} mins.')

# 4 测试
## 4.1 载入模型参数和测试集

In [ ]:
model = CNN(class_num=y_test.shape[1])
model.load_state_dict(torch.load('model_path\\best_network.pth', map_location=device),
                                                          strict=False)
test_loader = get_dataloader(X_test, y_test, batch_size=128)

In [ ]:
test_bar = tqdm(enumerate(test_loader), total=len(test_loader), ncols=50)  # 创建进度条
num_correct
y_predicted = []
for _, (batch_x, batch_y) in test_bar:
    test_outputs = model(batch_x.to(device))
    loss = criterion(test_outputs, batch_y.to(device))
    optimizer.zero_grad()
    # predicted_labels = (test_outputs >= val_threshold).float().cpu().numpy()
    # num_correct += np.count_nonzero(predicted_labels == batch_y)
    y_predicted.append(predicted_labels)
y_predicted = np.vstack(y_predicted)

## 4.2 模型性能可视化  
**性能指标：**  
我目前只想到了误差